In [28]:
import pandas as pd
import time
import polars as pl
import numpy as np

## Abriendo con polars

In [29]:
cur_time = time.time()
pl_df = pl.read_parquet("parquet/sitios1.parquet")
print(f"Tiempo en leer usando polars = {round((time.time()-cur_time),2)} segundos")
pl_df.shape

Tiempo en leer usando polars = 4.05 segundos


(3025011, 15)

## Elimino duplicados en name

In [30]:
pl_df = pl_df.filter(pl_df.select(pl.col('name')).is_duplicated())
pl_df.shape

(870913, 15)

## Aqui empiezo la limpieza

## Establecimientos Gastronomicos de la Florida

In [31]:
def busca_zip(x):
    if x.strip()[len(x)-15:].upper() == ',UNITED STATES':
        x = x.strip()[:len(x)-15]
    return x.strip()[len(x)-5:]  

def busca_state(x):
    if x.strip()[len(x)-15:].upper() == ',UNITED STATES':
        x = x.strip()[:len(x)-15]
    x = x.strip()[:len(x)-6]
    return x.strip()[len(x)-2:] 

def busca_street(x):
    try:
        x1 = x[x.index(',')+2:]
        return x1[:x1.index(',')]+' FL USA'
    except:
        return x

In [35]:
cur_time = time.time()
pl_df1 = (
    pl_df.lazy()
    # Solo restaurant, bakery, desserts, pastry 
    .filter(pl.col('category').apply(lambda x: True if ('RESTAURANT' in ''.join(x).upper() or
                                                        'BAKERY' in ''.join(x).upper() or
                                                        'DESSERTS' in ''.join(x).upper() or
                                                        'PASTRY' in ''.join(x).upper() 
                                                       ) else False))
    # Creo columnas nuevas para ML
    .with_columns(pl.col('category').apply(lambda x: ''.join(x)))
    .with_columns(pl.col('category').apply(lambda x: 1 if 'RESTAURANT' in ''.join(x).upper() else 0).alias('restaurant'))
    .with_columns(pl.col('category').apply(lambda x: 1 if 'BAKERY' in ''.join(x).upper() else 0).alias('bakery'))
    .with_columns(pl.col('category').apply(lambda x: 1 if 'DESSERTS' in ''.join(x).upper() else 0).alias('dessert'))
    .with_columns(pl.col('category').apply(lambda x: 1 if 'PASTRY' in ''.join(x).upper() else 0).alias('pastry')) 
    .with_columns(pl.col('state').apply(lambda x: 1 if 'PERMANENTLY CLOSED' in x.upper() else 0).alias('closed')) 
    .with_columns(pl.col('address').apply(busca_street).alias('street'))
    # Solo los del estado de la Florida
    .with_columns([
        pl.col('address').apply(busca_zip).alias('zip'),
        pl.col('address').apply(busca_state).alias('st'),
        pl.col('address').apply(lambda x: x.split(',')[len(x.split(','))-2].strip().upper()).alias('City')    
    ])
    # Creo una columna que sea una copia de state para trabajarla y sacarle los horarios
    .with_columns(pl.col('state').alias('horario'))
    .filter(pl.col('st')=='FL')
    # Eliminacion de columnas:        
    #.select(pl.exclude(['gmap_id','description','price','hours','MISC','relative_results','state']))  
    .collect()
)    
print(f"{pl_df1.shape[0]} records que quedan, tiempo usado = {round((time.time()-cur_time),2)} segundos")

4127 records que quedan, tiempo usado = 4.21 segundos


### Verifique varios casos de valores NaN en la columna closed y todos estaban 'Permanently closed'
### asi que todos los valores NaN de la columna closed los pongo en 1

In [36]:
pl_df2 = pl_df1.with_columns(pl.col('closed')).fill_null(1)

In [38]:
pl_df2.head(2)

name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url,restaurant,bakery,dessert,pastry,closed,street,zip,st,City,horario
str,str,str,str,f32,f32,str,f32,i16,str,f64,f64,str,str,str,i64,i64,i64,i64,i64,str,str,str,str,str
"""Cape Seafood S...","""Cape Seafood S...","""0x88db4147b1d9...",null,26.641376,-81.940544,"""Restaurant""",5.0,1,"""$$""",1.0,1.0,null,null,"""https://www.go...",1,0,0,0,1,"""603 Del Prado ...","""33990""","""FL""","""CAPE CORAL""",null
"""Fresh Point Co...","""Fresh Point Co...","""0x88c2e4e34f1e...",null,27.867489,-82.702972,"""Buffet restaur...",5.0,2,null,1.0,1.0,"""Permanently cl...","""0x88c2e5593fbd...","""https://www.go...",1,0,0,0,1,"""10525 US-19 FL...","""33782""","""FL""","""PINELLAS PARK""","""Permanently cl..."


### Hago cast a la columna zip a Int64

In [39]:
pl_df2 = pl_df2.with_columns(pl.col("zip").cast(pl.Int64))

### Creo las columnas open y close que van a tener los horarios de apertura y cierre, 
### la informacion sale de la columna state

In [40]:
def busca_close(x):
    if x == 'Permanently closed': return x
    if x[:13] == 'Open ⋅ Closes': return x[14:]
    if x[:8] == 'Closed ⋅': return None
    if x == 'Open 24 hours': return x
    if x[:12] == 'Opens soon ⋅': return None
    if x[:14] == 'Closes soon ⋅ ': return x[14:][:7]
    if x == 'Open now': return None
    if x == 'Temporarily closed': return x
    if x == 'Closed': return x
    return 'nada'

In [41]:
def busca_open(x):
    if x == 'Permanently closed': return x

    if x[:13] == 'Closes soon ⋅':
        try:
            return texto[14:][6:][texto[14:][6:].index('Opens')+6:]
        except:
            try:
                return texto[14:][6:][texto[14:][6:].index('Reopens')+8:]
            except:
                return None
    if x[:12] == 'Opens soon ⋅': return x[13:]
    if x[:8] == 'Open now': return None
    if x[:6] == 'Open ⋅': return None
    if x[:14] == 'Closed ⋅ Opens': return x[15:] 
    if x == 'Open 24 hours': return x 
    if x == 'Temporarily closed': return x
    if x == 'Closed': return x 
    return 'nada'

In [42]:
def rectifico_close(x):
    if x[:5] == '1AM ⋅': return '1AM'
    if x[:5] == '2AM ⋅': return '2AM'
    if x[:5] == '2PM ⋅': return '2PM'
    if x[:5] == '3PM ⋅': return '3PM'
    if x[:5] == '4PM ⋅': return '4PM'
    if x[:5] == '5PM ⋅': return '5PM'
    if x[:5] == '6PM ⋅': return '6PM'
    if x[:5] == '7PM ⋅': return '7PM'
    if x[:5] == '8PM ⋅': return '8PM'
    if x[:5] == '8AM ⋅': return '8AM'
    if x[:5] == '9PM ⋅': return '9PM'
    if x[:6] == '10PM ⋅': return '10PM'
    if x[:6] == '11AM ⋅': return '11AM'
    if x[:6] == '11PM ⋅': return '11PM'
    if x[:6] == '12AM ⋅': return '12AM'
    return x

In [43]:
pl_df3 = pl_df2.with_columns([
    pl.col('state').apply(busca_open).alias('open'),
    pl.col('state').apply(busca_close).alias('close')
])
pl_df3 = pl_df3.with_columns([
   pl.col('close').apply(rectifico_close)
])
pd_df3 = pl_df3.to_pandas()
pl_df3.shape

(4127, 27)

In [44]:
pl_df3.head(2)

name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url,restaurant,bakery,dessert,pastry,closed,street,zip,st,City,horario,open,close
str,str,str,str,f32,f32,str,f32,i16,str,f64,f64,str,str,str,i64,i64,i64,i64,i64,str,i64,str,str,str,str,str
"""Cape Seafood S...","""Cape Seafood S...","""0x88db4147b1d9...",null,26.641376,-81.940544,"""Restaurant""",5.0,1,"""$$""",1.0,1.0,null,null,"""https://www.go...",1,0,0,0,1,"""603 Del Prado ...",33990,"""FL""","""CAPE CORAL""",null,null,null
"""Fresh Point Co...","""Fresh Point Co...","""0x88c2e4e34f1e...",null,27.867489,-82.702972,"""Buffet restaur...",5.0,2,null,1.0,1.0,"""Permanently cl...","""0x88c2e5593fbd...","""https://www.go...",1,0,0,0,1,"""10525 US-19 FL...",33782,"""FL""","""PINELLAS PARK""","""Permanently cl...","""Permanently cl...","""Permanently cl..."


## Agrego columnas de county y city que pueden servir al grupo de Analitics y el de Machine Learning

In [46]:
pl_zip = pl.read_csv('csv/zip_florida.csv')
pl_zip.shape

(994, 6)

In [47]:
pl_zip .head(2)

zip,Zipcode name,City,State,County Name,population
i64,str,str,str,str,i64
32024,"""""LAKE CITY. FL...","""LAKE CITY""","""FL""","""COLUMBIA""",12329
32025,"""""LAKE CITY. FL...","""LAKE CITY""","""FL""","""COLUMBIA""",12329


In [48]:
pl_df3 = pl_df3.join(pl_zip.select(['zip','County Name']), on='zip', how="left" )
pl_df3.shape

(4127, 28)

In [50]:
pl_df3.head(2)

name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url,restaurant,bakery,dessert,pastry,closed,street,zip,st,City,horario,open,close,County Name
str,str,str,str,f32,f32,str,f32,i16,str,f64,f64,str,str,str,i64,i64,i64,i64,i64,str,i64,str,str,str,str,str,str
"""Cape Seafood S...","""Cape Seafood S...","""0x88db4147b1d9...",null,26.641376,-81.940544,"""Restaurant""",5.0,1,"""$$""",1.0,1.0,null,null,"""https://www.go...",1,0,0,0,1,"""603 Del Prado ...",33990,"""FL""","""CAPE CORAL""",null,null,null,"""LEE"""
"""Fresh Point Co...","""Fresh Point Co...","""0x88c2e4e34f1e...",null,27.867489,-82.702972,"""Buffet restaur...",5.0,2,null,1.0,1.0,"""Permanently cl...","""0x88c2e5593fbd...","""https://www.go...",1,0,0,0,1,"""10525 US-19 FL...",33782,"""FL""","""PINELLAS PARK""","""Permanently cl...","""Permanently cl...","""Permanently cl...","""PINELLAS"""


## Agrego avg_rating, num_of_reviews de review_Fl

In [69]:
pl_review = pl.read_csv('csv/review_FL.csv')
pl_review.shape

(248691, 6)

In [80]:
pl_review = pl_review.filter((pl.col('rating')>= 1.0) & (pl.col('rating') <= 5.0))

## Agrego a pl_review los campost rating1, rating2, rating3, rating4, rating5

In [81]:
pl_review = pl_review.with_columns([
    pl.col('rating').apply(lambda x: 1 if x == 1 else 0).alias('rating1'),
    pl.col('rating').apply(lambda x: 1 if x == 2 else 0).alias('rating2'),
    pl.col('rating').apply(lambda x: 1 if x == 3 else 0).alias('rating3'),
    pl.col('rating').apply(lambda x: 1 if x == 4 else 0).alias('rating4'),
    pl.col('rating').apply(lambda x: 1 if x == 5 else 0).alias('rating5')
])

In [82]:
pl_review.head()

name,name_right,rating,text,date,zip,rating1,rating2,rating3,rating4,rating5
str,str,f64,str,str,i64,i64,i64,i64,i64,i64
"""33 Below Icecr...","""Veronica Franc...",5.0,"""Went for the S...","""2019-06-12""",33323,0,0,0,0,1
"""33 Below Icecr...","""Sonia Bauer""",5.0,"""Delicious!!!!!...","""2018-11-30""",33323,0,0,0,0,1
"""33 Below Icecr...","""Marc V Nation""",4.0,null,"""2019-08-15""",33323,0,0,0,1,0
"""33 Below Icecr...","""Jill Reinhart""",5.0,"""Customer servi...","""2018-07-08""",33323,0,0,0,0,1
"""33 Below Icecr...","""Carvalee B""",3.0,null,"""2019-09-27""",33323,0,0,1,0,0


In [83]:
pl_review = (pl_review.groupby('name').agg([
    pl.col('rating').mean().alias('avg_rating'),
    pl.col('text').count().alias('num_of_reviews'),
    pl.col('rating1').sum(),
    pl.col('rating2').sum(),
    pl.col('rating3').sum(),
    pl.col('rating4').sum(),
    pl.col('rating5').sum()
]).with_columns(pl.col('avg_rating').apply(lambda x: round(x,2)))
)

In [84]:
pl_review.head()

name,avg_rating,num_of_reviews,rating1,rating2,rating3,rating4,rating5
str,f64,u32,i64,i64,i64,i64,i64
"""Golden Krust""",4.17,206,14,6,24,50,112
"""Golden Wok""",4.08,76,5,5,11,13,42
"""Los Compadres""",4.39,36,0,1,4,11,20
"""Mi Tierra Mexi...",4.56,1344,30,28,74,235,975
"""Smoothie King""",4.28,1829,94,70,178,376,1107


### Hago el join entre sitios y pl_review para sustituir avg_rating y num_of_reviews que venian
### originalmente en sitios por los valores calculados a partir de la tabla reviews

In [85]:
#pl_df3.select(pl.exclude(['avg_rating','num_of_reviews'])).join(pl_review,on='name', how="left")
pl_df3 = pl_df3.join(pl_review,on='name', how="left").with_columns([
    pl.col('avg_rating').cast(pl.Float32),
    pl.col('avg_rating_right').cast(pl.Float32)
])
pl_df3.shape

(4127, 35)

## Le agrego latitude1 y longitude1 que estan en coordenadas_FL

In [75]:
pl_coordenadas = pl.read_csv('csv/coordenadas_FL.csv')
pl_coordenadas.shape

(4127, 3)

In [78]:
pl_coordenadas.head()

name,latitude1,longitude1
str,f64,f64
"""Cape Seafood S...",26.640758,-81.940728
"""Fresh Point Co...",27.867496,-82.703016
"""Oneyda's Baker...",26.270501,-81.789587
"""Annie's Bake S...",25.717426,-80.361958
"""Hot Box""",0.0,0.0


In [79]:
#pl_df3.join(pl_coordenadas,on='name', how="inner")
#pl_df3.shape

## Esto es lo que voy a salvar

In [86]:
pl_df3.head()

name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url,restaurant,bakery,dessert,pastry,closed,street,zip,st,City,horario,open,close,County Name,avg_rating_right,num_of_reviews_right,rating1,rating2,rating3,rating4,rating5
str,str,str,str,f32,f32,str,f32,i16,str,f64,f64,str,str,str,i64,i64,i64,i64,i64,str,i64,str,str,str,str,str,str,f32,u32,i64,i64,i64,i64,i64
"""Cape Seafood S...","""Cape Seafood S...","""0x88db4147b1d9...",null,26.641376,-81.940544,"""Restaurant""",5.0,1,"""$$""",1.0,1.0,null,null,"""https://www.go...",1,0,0,0,1,"""603 Del Prado ...",33990,"""FL""","""CAPE CORAL""",null,null,null,"""LEE""",null,null,null,null,null,null,null
"""Fresh Point Co...","""Fresh Point Co...","""0x88c2e4e34f1e...",null,27.867489,-82.702972,"""Buffet restaur...",5.0,2,null,1.0,1.0,"""Permanently cl...","""0x88c2e5593fbd...","""https://www.go...",1,0,0,0,1,"""10525 US-19 FL...",33782,"""FL""","""PINELLAS PARK""","""Permanently cl...","""Permanently cl...","""Permanently cl...","""PINELLAS""",4.34,813,29,22,85,187,489
"""Oneyda's Baker...","""Oneyda's Baker...","""0x88dae191ee50...",null,26.154755,-81.790527,"""Bakery,Deli""",4.6,19,"""$""",1.0,1.0,"""Permanently cl...","""0x88dae1997e12...","""https://www.go...",0,1,0,0,1,"""600 Goodlette-...",34102,"""FL""","""NAPLES""","""Permanently cl...","""Permanently cl...","""Permanently cl...","""COLLIER""",null,null,null,null,null,null,null
"""Annie's Bake S...","""Annie's Bake S...","""0x88d9c754413f...",null,25.717417,-80.361946,"""Bakery,Gift ba...",4.3,3,null,1.0,1.0,"""Open ⋅ Closes ...","""0x88d9c75ec79f...","""https://www.go...",0,1,0,0,0,"""10331 SW 54th ...",33165,"""FL""","""MIAMI""","""Open ⋅ Closes ...",null,"""4PM""","""MIAMI-DADE""",4.33,48,4,1,3,7,33
"""Hot Box""","""Hot Box, 1-99 ...","""0x8890b9241e70...",null,30.391411,-87.26722,"""Restaurant""",4.2,5,null,1.0,1.0,"""Permanently cl...",null,"""https://www.go...",1,0,0,0,1,"""1-99 Kinkaid S...",32507,"""FL""","""PENSACOLA""","""Permanently cl...","""Permanently cl...","""Permanently cl...","""ESCAMBIA""",null,null,null,null,null,null,null


## Salvo Datos Establecimientos Gastronomicos de la Florida 

In [87]:
cur_time = time.time()
pl_df3.write_parquet("parquet/sitios_FL.parquet")
pl_df3.write_csv("csv/sitios_FL.csv")
pl_df3.write_csv("PG_Google_Maps/Ornaldo/sitios_FL.csv")
print(f"tiempo usado = {round((time.time()-cur_time),2)} segundos")

tiempo usado = 0.1 segundos
